In [ ]:
import sys
import argparse
import random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

import nltk 
from nltk.tokenize import word_tokenize 
nltk.download("stopwords")
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# stopwords after removing negation words
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'ma', 'shan', 'won',]


In [ ]:
def identity(x):
    '''Dummy function that just returns the input'''
    return x
def preprocess_data(data):
    '''Preprocess the data by tokenization and removing stopwords without negation'''
    processed_data = word_tokenize(data)
    processed_data = [word for word in processed_data if word not in stopwords]

    return processed_data

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Keypoints/intermediary_texts/inter-text_from_T5-small_temp_1_full_train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Keypoints/intermediary_texts/inter-text_from_T5-small_temp_1_full_test.csv")

In [ ]:
X_train = [row['argument']+row['key_point']+row['intermediary_text'] for index,row in train_df.filter(['argument', 'key_point', 'intermediary_text'], axis=1).iterrows()]
Y_train = train_df.filter(['label'], axis=1)

X_test = [row['argument']+row['key_point']+row['intermediary_text'] for index,row in test_df.filter(['argument', 'key_point', 'intermediary_text'], axis=1).iterrows()]
Y_test = test_df.filter(['label'], axis=1)

In [ ]:
X_train = [preprocess_data(i) for i in X_train]
X_test = [preprocess_data(i) for i in X_test]

In [ ]:
vec = TfidfVectorizer(preprocessor=identity, tokenizer=identity)

# NB
classifier1 = Pipeline([('vec', vec), ('cls', MultinomialNB())])

# SVM
classifier2 = Pipeline([('vec', vec), ('cls', svm.SVC(kernel='linear', C = 1.0))])

# DT
classifier3 = Pipeline([('vec', vec), ('cls', DecisionTreeClassifier(random_state=0))])

classifier1.fit(X_train, Y_train)
classifier2.fit(X_train, Y_train)
classifier3.fit(X_train, Y_train)

Y_pred_test1 = classifier1.predict(X_test)
Y_pred_test2 = classifier2.predict(X_test)
Y_pred_test3 = classifier3.predict(X_test)

print("NB:",classification_report(Y_test, Y_pred_test1, digits = 3))
print("SVM:",classification_report(Y_test, Y_pred_test1, digits = 3))
print("DT:",classification_report(Y_test, Y_pred_test1, digits = 3))
